<a href="https://colab.research.google.com/github/klushcheva/ai_generated_text_detection/blob/main/many_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    GenerationConfig, AutoConfig
)
from concurrent.futures import ThreadPoolExecutor
import numpy as np
from tqdm import tqdm
import pandas as pd
from random import choice

In [ ]:
saiga_yandexgpt_8b = "C:/Users/KseniaLuschevaExt/Documents/models/saiga_yandexgpt_8b"
rut5_base_headline_gen_telegram = "C:/Users/KseniaLuschevaExt/Documents/models/rut5_base_headline_gen_telegram"
rugpt3large_based_on_gpt2 = "C:/Users/KseniaLuschevaExt/Documents/models/rugpt3large_based_on_gpt2"

In [87]:
from transformers import (
    AutoModelForCausalLM,  # For GPT-like models
    AutoModelForSeq2SeqLM,  # For T5-like models
    AutoTokenizer,
    AutoConfig
)

# Configuration
MODEL_POOL = [
    {
        "name": "ai-forever/rugpt3large_based_on_gpt2",
        "type": "base",
        "temperature": 0.9,
        "model_class": "causal"
    },
    {
        "name": "IlyaGusev/rut5_base_headline_gen_telegram",
        "type": "news",
        "temperature": 0.5,
        "model_class": "seq2seq"
    }
]

class ModelWrapper:
    def __init__(self, model_info):
        self.device = "cpu"
        self.tokenizer = AutoTokenizer.from_pretrained(model_info["name"])

        # Get config first to determine architecture
        config = AutoConfig.from_pretrained(model_info["name"])

        # Select appropriate model class
        if model_info["model_class"] == "causal":
            self.model = AutoModelForCausalLM.from_pretrained(
                model_info["name"],
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                device_map="auto"
            )
        elif model_info["model_class"] == "seq2seq":
            self.model = AutoModelForSeq2SeqLM.from_pretrained(
                model_info["name"],
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                device_map="auto"
            )
        else:
            raise ValueError(f"Unknown model class: {model_info['model_class']}")

        self.model.eval()
        self.model_type = model_info["type"]
        self.default_params = {
            "max_new_tokens": 500,
            "do_sample": True,
            "top_p": 0.9,
            "temperature": model_info["temperature"]
        }

    def generate(self, prompt, **kwargs):
        try:
            # Merge default params with any overrides
            params = {**self.default_params, **kwargs}

            if isinstance(self.model, AutoModelForSeq2SeqLM):
                # T5-style models
                inputs = self.tokenizer(
                    prompt,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=512
                ).to(self.device)

                outputs = self.model.generate(
                    **inputs,
                    **params
                )
            else:  # Causal LM
                inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
                outputs = self.model.generate(
                    **inputs,
                    **params,
                    pad_token_id=self.tokenizer.eos_token_id
                )

            return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        except Exception as e:
            print(f"Error in {self.model.config.name_or_path}: {str(e)}")
            return None

In [88]:
topics = [
    "изменение климата", "искусственный интеллект", "исследование космоса",
    "квантовые вычисления", "древние цивилизации", "технологии будущего",
    "нейробиология", "криптовалюта", "древние мифы", "виртуальная реальность",
    "синтетическая биология", "черные дыры", "сознание", "постапокалиптические общества",
    "океанография", "путешествия во времени", "лингвистика", "нанотехнологии", "анализ сновидений",
    "межзвездная дипломатия", "дружба с животными", "преподавание русского языка", "спортивные достижения",
    "политические конфликты", "любовь между людьми"
]

tones = [
    "нейтральном", "восторженном", "скептическом", "юмористическом",
    "аналитическом", "саркастичном", "меланхоличном", "вдохновляющем", "драматическом",
    "причудливом", "мрачном", "оптимистичном", "пессимистичном", "загадочном",
    "авторитетном", "неформальном", "романтическом", "отстранённый"
]

actions = [
    "объяснить", "описать", "обсудить", "написать рассказ о",
    "проанализировать", "сравнить и сопоставить", "предсказать будущее",
    "критиковать", "представить мир, где", "подвести итог",
    "защитить", "оспаривать", "переосмыслить", "придумать новую теорию о",
    "пародировать", "взять интервью у эксперта по", "написать новостной репортаж о",
    "создать диалог о", "перечислить плюсы и минусы", "исследовать этические аспекты", "написать научную статью о"
]

styles = ["формальный", "деловой", "шуточный", "школьный", "творческий", "публицистический"]

roles = ["профессиональный писатель", "школьник", "студент с плохими оценками", "человек с дислексией", "маленькая девочка", "писатель-фантаст",
        "писатель-романист", "журналист", "обычный пользователь интернета", "учитель химии и биологии", "малограмотный крестьянин 1902 года рождения"]

In [8]:
import random

In [89]:
def generate_prompts(num_prompts=10):
    prompts = []
    for _ in range(num_prompts):
        style = random.choice(styles)
        topic = random.choice(topics)
        tone = random.choice(tones)
        action = random.choice(actions)

        prompt = (
            f"Напиши текст в {style} стиле, в тональности '{tone}'. "
            f"Текст должен {action} тему {topic}'. "
            f"Убедись, что текст выражает стиль {style} достоверно."
        )
        prompts.append(prompt)
    return prompts

In [91]:
import concurrent.futures
from functools import partial

def generate_corpus(num_texts, output_csv="generated_corpus.csv", max_workers=4, batch_size=8):
    """
    Generates corpus using diverse prompts and parallel model execution.

    Args:
        num_texts (int): Total texts to generate
        output_csv (str): Output file path
        max_workers (int): Parallel threads (match GPU count)
        batch_size (int): Texts per parallel batch
    """
    # 1. Initialize models
    models = {}
    for model_info in MODEL_POOL:
        try:
            wrapper = ModelWrapper(model_info)
            models[model_info["name"]] = wrapper
            print(f"✅ Loaded {model_info['name']}")
        except Exception as e:
            print(f"❌ Failed to load {model_info['name']}: {str(e)}")

    if not models:
        raise ValueError("No models available!")

    def create_prompt():
        style = random.choice(styles)
        topic = random.choice(topics)
        tone = random.choice(tones)
        action = random.choice(actions)
        role = random.choice(roles)

        return (
            f"Представь, что ты - {role}. Требуется {action} тему '{topic}' "
            f"в {tone} тоне, используя {style} стиль. Текст должен "
            f"быть законченным произведением из 300+ слов.\n\n"
            f"Текст должен начинаться сразу с раскрытия темы:\n"
        ), style

    all_prompts, all_styles = zip(*[create_prompt() for _ in range(num_texts)])

    def process_batch(batch_prompts, batch_styles):
        batch_results = []
        for prompt, style in zip(batch_prompts, batch_styles):
            model_name = weighted_model_choice(models, style)
            model = models[model_name]

            try:
                # Critical generation parameters to prevent instruction copying
                gen_params = {
                    "temperature": 0.8,
                    "top_p": 0.95,
                    "repetition_penalty": 1.2,
                    "no_repeat_ngram_size": 3,
                    "begin_suppress_tokens": [model.tokenizer.eos_token_id],
                    "max_new_tokens": 700  # For 300+ word texts
                }

                # Generate with forced content start
                full_prompt = prompt + "[НАЧАЛО ТЕКСТА]\n"
                text = model.generate(full_prompt, **gen_params)

                # Extract only the generated content
                if text:
                    # Remove the prompt and any trailing instructions
                    text = text.split("[НАЧАЛО ТЕКСТА]")[-1].strip()
                    text = text.split("Инструкция:")[0].strip()
                    text = text.split("Примечание:")[0].strip()

                    # Verify quality
                    if len(text.split()) >= 300:
                        batch_results.append({
                            "text": text,
                            "prompt": prompt,
                            # [rest of your metadata...]
                        })
            except Exception as e:
                print(f"Generation error: {str(e)}")
        return batch_results

    corpus = []
    with tqdm(total=num_texts, desc="Generating corpus") as pbar:
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = []
            for i in range(0, num_texts, batch_size):
                batch_p = all_prompts[i:i + batch_size]
                batch_s = all_styles[i:i + batch_size]
                futures.append(executor.submit(process_batch, batch_p, batch_s))

            for future in concurrent.futures.as_completed(futures):
                batch_results = future.result()
                corpus.extend(batch_results)
                pbar.update(len(batch_results))

    # 6. Create DataFrame and ensure all columns exist
    df = pd.DataFrame(corpus)

    # Ensure required columns exist
    if 'length' not in df.columns:
        if 'text' in df.columns:
            df['length'] = df['text'].str.split().str.len()
        else:
            raise ValueError("Generated corpus is missing both 'length' and 'text' columns")

    # Apply filters
    df = df.drop_duplicates(subset=['text'])
    df = df[df['length'] >= 150]  # Adjusted from 150 to match earlier check

    # Add metadata
    df['generation_date'] = pd.Timestamp.now()
    df['word_count'] = df['text'].str.split().str.len()  # Exact word count

    # Save results
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')

    print(f"\nSuccessfully generated {len(df)} texts")
    print("Columns available:", df.columns.tolist())
    return df

def weighted_model_choice(models, style):
    """Select model with preference for style matching"""
    style_preference = {
        "formal": ["sberbank-ai/rugpt3large_based_on_gpt2"],
        "creative": ["IlyaGusev/saiga_yandexgpt_8b", "IlyaGusev/rut5_base_headline_gen_telegram"]
    }

    # Try preferred models first
    for model_name in style_preference.get(style, []):
        if model_name in models:
            return model_name

    # Fallback to any available model
    return random.choice(list(models.keys()))

In [ ]:
corpus_df = generate_corpus(
    num_texts=50,
    batch_size=4  # Texts to generate in paralle
)

✅ Loaded ai-forever/rugpt3large_based_on_gpt2


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/977M [00:00<?, ?B/s]

✅ Loaded IlyaGusev/rut5_base_headline_gen_telegram


Generating corpus:   0%|          | 0/50 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

Generating corpus:  10%|█         | 5/50 [22:10<2:44:00, 218.69s/it]

In [67]:
df = pd.read_csv('/content/generated_corpus.csv')

In [68]:
df.head()

,text,prompt,length,generation_date,word_count
0,"В этом тексте я расскажу о том, что такое крип...",Ты - профессиональный писатель. обсудить тему ...,454,2025-04-01 15:37:43.333185,454
1,Внимание! К тексту нужно прикрепить видео или ...,Ты - учитель химии и биологии. проанализироват...,436,2025-04-01 15:37:43.333185,436
2,Каков состав вещества звезды? В каких условиях...,Ты - учитель химии и биологии. критиковать тем...,506,2025-04-01 15:37:43.333185,506
3,- Океанариум и аквапарк на Пхукете\n- Вулканы ...,Ты - профессиональный писатель. написать новос...,468,2025-04-01 15:37:43.333185,468
4,"<< <ПРОДОЛЖЕНИЕ >>>\n""Нефть? Не-а, крипта... У...",Ты - учитель химии и биологии. оспаривать тему...,415,2025-04-01 15:37:43.333185,415


In [84]:
df.iloc[3, 0]

'- Океанариум и аквапарк на Пхукете\n- Вулканы острова Самуи (Пхукет) \n\nИ далее следует описание морского побережья:\nЯ хочу новый дом! Я буду жить не в офисе, а в своем доме! Когда я выйду замуж? Скоро! А пока мы будем заниматься строительством! Как вам такая перспектива? Ведь это же так здорово, правда? :) И вообще идея шикарная... Почему бы нам тоже не построить свой дом?! Пусть даже в Испании ;) Вот скажите мне, если вдруг кто-то решил переехать в Испанию со своим домом в пригороде Мадрида или Барселоны, то почему бы ему просто не сделать его самостоятельно? Не платить агентству недвижимости, самому возводить стены, покупать все необходимое для дома и тд. Короче говоря, что будет дешевле, быстрее, да еще у вас будет куча времени на всех этапах строительства, причем за счет того, что вы будете строить ваш дом сами? \nВот как только я подумаю об этом, сразу же куплю себе красивый домик и начну строить уже внутри него. Главное, чтобы был хороший участок земли под застройку :D\nА пок